In [2]:
import pandas as pd
import os
import re
import romkan
import unicodedata
import csv
from collections import Counter

In [44]:
# get irregular words from ../data/irregular/irregular.csv and create a dict to avoid those words

In [8]:
#kanji[kanji.character == '広']

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
858,広,263.0,"[wide, broad, spacious]",[こう],"['ひろ.い', 'ひろ.まる', 'ひろ.める', 'ひろ.がる', 'ひろ.げる']",[],[ひろ],"[ひろい, ひろめる, ひろがる, ひろげる, ひろまる]","[ひろ, ひろい, ひろめる, ひろがる, ひろげる, ひろまる, こう]"


In [9]:
#kanji[kanji.character == '大']

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
1762,大,7.0,"[large, big]","[だい, たい]","['おお-', 'おお.きい', '-おお.いに']","[うふ, お, おう, た, たかし, とも, はじめ, ひろ, ひろし, まさ, まさる,...",[おお],"[おお, おおいに, おおきい]","[おお, おお, おおいに, おおきい, だい, たい]"


In [79]:
propernouns = pd.read_csv('../data/propernouns_subset.csv')
kanji = pd.read_csv('../data/kanjidic2.csv')
vocab = pd.read_csv('../data/created/JMdict_e.csv', sep='\t')

In [80]:
#remove all entries with Roman characters and numbers in the name
propernouns = propernouns[propernouns.kanji.map(lambda x: True if re.match('[０-９]+|[ａ-ｚＡ-Ｚ]+|ゝ|〆', x) == None else False)]

In [81]:
# Run once
del propernouns['index']
del kanji['Unnamed: 0']
propernouns.pronunciation = propernouns.pronunciation.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
kanji.on = kanji.on.map(lambda x: re.findall("""[\u3040-\u309f]+""", romkan.to_hiragana(romkan.to_roma(x))))
##############################
#kanji.kun = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x))))) # THIS THREW AWAY GOOD INFO
##############################
kanji['kun1'] = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x)))))
kanji['kun2'] = kanji.kun.map(lambda x: list(set(map(lambda y: ''.join(y.split('.')),re.findall("""[\u3040-\u309f.]+""", x)))))
# I should also include the kun without splitting somehow... aka shi.ku = shi and shiku
kanji.nanori = kanji.nanori.map(lambda x: re.findall("""[\u3040-\u309f]+""", x))
kanji.meaning = kanji.meaning.map(lambda x: re.findall("""[a-zA-Z-]+ *[a-zA-Z-]+""", x))
#char_ser = pd.Series(kanji.on.values,index=kanji.character).to_dict()
kanji['onkun'] = kanji.kun1 + kanji.kun2 + kanji.on
#kanji.onkun.map(lambda x: list(set(x))) ########## NOW MAKE SURE ALL ARE SET TO LISTS SO THERE ARE NO DUPLICATES
char_ser = pd.Series(kanji.onkun.values,index=kanji.character).to_dict()
kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
0,亜,1509.0,"[Asia, rank next, come after, -ous]",[あ],['つ.ぐ'],"[や, つぎ, つぐ]",[つ],[つぐ],"[つ, つぐ, あ]"
1,唖,NaN,"[mute, dumb]","[あ, あく]",['おし'],[],[おし],[おし],"[おし, おし, あ, あく]"
2,娃,NaN,[beautiful],"[あ, あい, わ]",['うつく.しい'],[い],[うつく],[うつくしい],"[うつく, うつくしい, あ, あい, わ]"
3,阿,1126.0,"[Africa, flatter, fawn upon, corner, nook, rec...","[あ, お]","['おもね.る', 'くま']","[ほとり, あず, あわ, おか, きた, な]","[おもね, くま]","[おもねる, くま]","[おもね, くま, おもねる, くま, あ, お]"
4,哀,1715.0,"[pathetic, grief, sorrow, pathos, pity, sympat...",[あい],"['あわ.れ', 'あわ.れむ', 'かな.しい']",[],"[あわ, かな]","[あわれ, かなしい, あわれむ]","[あわ, かな, あわれ, かなしい, あわれむ, あい]"


In [10]:
def unvoice(char):
    if not ('\u3041' <= char <= '\u3096') or len(char) > 1 or not unicodedata.decomposition(char):
        return char
    return unicodedata.normalize('NFD', char)[0]

In [11]:
# if prons[-1] == ん: prons = [prons,prons[:-1]]
# if 
# write pseudocode

#kanji_pronunciations = dict()

def recurs(kanji, word, pron_list = [], done = False): # create a dictionary for kanji: set of pronunciations
    if len(pron_list) == len(kanji) or len(word) == 0: # check for 々 and repeat past syllable (unsounded)
        return pron_list, False
    cur = kanji[len(pron_list)]
    if re.match("""[\u3040-\u30ff]""", cur) != None: #should we romkan to hiragana?? ALSO HYPHEN
        if cur != word[0]:
            return pron_list, False
        prons = [cur]
    elif cur == '々':
        if unvoice(pron_list[-1]) == unvoice(word[:len(pron_list[-1])]): # Sou Zou -- didn't catch!!
            pron_list.append(word[:len(pron_list[-1])])
            return pron_list, len(kanji) == len(pron_list)
        else:
            return None, False
    else:
        prons = list(set(char_ser.get(cur, []))) # [char_ser[cur]] ###.sort(key=len, reverse=True)
        if prons == []:
            return None, False
    for pron in prons:        # GET RID OF THIS?
        if pron[-1] == 'ん'and pron != 'ん':
            if isinstance(prons,str):
                prons = [pron, pron[:-1]]
            elif pron[:-1] not in prons:
                prons.append(pron[:-1])
    for pron in prons:
        wordmatch = word[:len(pron)] ###-1
        wordmatch = unvoice(wordmatch[0]) + wordmatch[1:]
        words = [wordmatch, word[:len(pron)]] ### -1
        #print(kanji, word, pron_list, words)
        if pron in words:
            pron_list.append(word[:len(pron)])
            word = word[len(pron):]
            if len(pron_list) == len(kanji) and len(word) == 0:
                return pron_list, True
            new_pron_list, done = recurs(kanji,word,pron_list,done)
            if done:
                return new_pron_list, done
            word = pron_list.pop() + word
    return None, False

# update with unvoice the first syllable of word


# PRONUNCIATION DOES NOT MATCH ON or KUN YOMIKATA... MOSTLY DROPPING LAST N??? -------------

In [12]:
list(set(char_ser['擽'])) #中腹 三日月
#'boy'[1:]
#recurs('相変わらず','あいかわらず',[])
#print(recurs('指輪','ゆびわ',[]))
#char_ser['一']#.sort(key=len, reverse=True)    # EITHER INCLUDE NANORI OR FIX THE ABOVE

['れき', 'ふき', 'らく', 'くすぐ', 'くすぐる', 'う', 'りゃく', 'うつ']

In [73]:
def kanji_counter(dataframe):
    true_cnt = 0
    false_cnt = 0
    false_dict = {}
    true_dict = {}

    kanji_pronunciations = dict()
    kanji_pron_counter = dict()

    for _, line in dataframe.iterrows():
        kanji = line.kanji
        word = line.pronunciation
        prons, _ = recurs(kanji,word,[])
        if prons == None or len(prons) == 0:
            false_cnt += 1
            false_dict[kanji] = word
        else:
            for i in range(len(kanji)):
                kanji_pronunciations[kanji[i]] = kanji_pronunciations.get(kanji[i], set())
                kanji_pronunciations[kanji[i]].add(prons[i])
                kanji_pron_counter[kanji[i]] = kanji_pron_counter.get(kanji[i], Counter())
                kanji_pron_counter[kanji[i]][prons[i]] += 1
            true_cnt += 1
            true_dict[kanji] = prons
    print(true_cnt, false_cnt) #~8% are irregular for jlpt N1 (jouyou vocab)
    print(kanji_pron_counter['高'])
#print(false_dict)
# 124,660 + 76,654 = 201,314
# 38% irregular. Much higher than ~8% irregular for everyday vocab.

In [53]:
vocab[1:].head()

,kanji,pronunciation,pos,meaning
1,仝,どうじょう,noun (common) (futsuumeishi),as above mark
2,々,くりかえし,noun (common) (futsuumeishi),repetition of kanji (sometimes voiced)
3,漢数字ゼロ,かんすうじゼロ,noun (common) (futsuumeishi),kanji zero
4,○,まる,noun (common) (futsuumeishi),circle (sometimes used for zero)
5,ＡＢＣ順,エービーシーじゅん,noun (common) (futsuumeishi),alphabetical order


In [52]:
propernouns.head()

,kanji,pronunciation,type,meaning
0,国労,こくろう,organization name,National Railway Workers' Union
86,ああ無情,ああむじょう,"work of art, literature, music, etc. name",Les Miserables (novel by Victor Hugo)
87,あいおい損害保険,あいおいそんがいほけん,company name,Aioi Insurance
88,あいか瞬,あいかしゅん,female given name or forename,Aika Shun (1984.12.8-)
89,あいがわゴルフ場,あいがわごるふじょう,place name,Aigawa golf links


In [55]:
vocab.columns = ['kanji','pronunciation','pos','meaning']
kanji_counter(vocab[4:])

127178 14648


In [48]:
propernouns.columns

Index(['kanji', 'pronunciation', 'type', 'meaning'], dtype='object')

In [61]:
boys = pd.read_csv('../data/Top10BoyNames.csv')
girls = pd.read_csv('../data/Top10GirlNames.csv')
surnames = pd.read_csv('../data/PopularSurnames.csv')

In [62]:
boys.columns = ['rank','kanji','pronunciation','frequency']
girls.columns = ['rank','kanji','pronunciation','frequency']
surnames.columns = ['rank','kanji','pronunciation','frequency']

In [63]:
#change pronunciation to hiragana
surnames['pronunciation'] = surnames['pronunciation'].map(lambda x: romkan.to_hiragana(re.sub('Ō|ō', 'ou', x)))
#convert string to number for frequency
surnames['frequency'] = surnames['frequency'].map(lambda x: int(x.replace(',','')))
surnames['pronunciation'] = surnames['pronunciation'].map(lambda x: x.split()[0])

boys['rank'] = boys['rank'].map(lambda x: re.search('[1-9]+', x).group())
boys['frequency'] = boys['frequency'].map(lambda x: re.search('[1-9]+', x).group())
boys['pronunciation'] = boys['pronunciation'].map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))

girls['rank'] = girls['rank'].map(lambda x: re.search('[1-9]+', x).group())
girls['frequency'] = girls['frequency'].map(lambda x: re.search('[1-9]+', x).group())
girls['pronunciation'] = girls['pronunciation'].map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))

In [66]:
kanji_counter(surnames)

88 12


In [75]:
kanji_counter(propernouns)

111764 89552
Counter({'たか': 261, 'こう': 108, 'だか': 53, 'ごう': 5, 'たかい': 1})


In [68]:
kanji_pron_counter['高']

Counter({'こう': 108, 'ごう': 5, 'たか': 261, 'たかい': 1, 'だか': 53})

In [85]:
kanji[kanji['character'] == '高']
#kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
911,高,65.0,"[tall, high, expensive]",[こう],"['たか.い', 'たか', '-だか', 'たか.まる', 'たか.める']","[か, こ, じょい, た, はか]","[だか, たか]","[だか, たか, たかめる, たかまる, たかい]","[だか, たか, だか, たか, たかめる, たかまる, たかい, こう]"


In [74]:
kanji_counter(vocab[4:])

127178 14648
Counter({'こう': 504, 'たか': 136, 'だか': 76})


In [87]:
kanji_pron_counter['高']

Counter({'こう': 108, 'ごう': 5, 'たか': 261, 'たかい': 1, 'だか': 53})

In [90]:
kanji_pronunciations

{'暗': {'あん', 'くら'},
 '韻': {'いん'},
 '来': {'き', 'ぎ', 'く', 'くる', 'ぐ', 'ぐる', 'こ', 'らい'},
 '狛': {'こま'},
 '就': {'しゅう', 'じゅ'},
 '秋': {'あき', 'しゅう', 'とき'},
 '戸': {'こ', 'と', 'ど'},
 '愛': {'あい', 'いと', 'お', 'かな', 'まな', 'め'},
 '役': {'やく'},
 '渭': {'い'},
 'ん': {'ん'},
 '戯': {'ぎ', 'げ'},
 '押': {'お', 'おう', 'おさ', 'おし', 'おす', 'おっ'},
 '干': {'かん', 'ひ', 'び', 'ほし', 'ぼ', 'ぼし'},
 '作': {'さ', 'さく', 'ざく', 'つくり', 'づくり'},
 '局': {'きょく', 'つぼね'},
 '仮': {'か', 'かり', 'がり', 'け'},
 '筬': {'おさ'},
 'で': {'で'},
 '酉': {'とり', 'どり'},
 '医': {'い'},
 '選': {'せん'},
 '齢': {'とし', 'れい'},
 '憂': {'う', 'ゆう'},
 '宮': {'きゅう', 'く', 'ぐ', 'ぐう', 'みや'},
 '仇': {'あだ', 'きゅう'},
 '葦': {'い'},
 '輪': {'りん', 'わ'},
 '奈': {'な'},
 '眉': {'び', 'み'},
 '抄': {'しょう'},
 'え': {'え'},
 '頌': {'しょう', 'じゅ'},
 '泡': {'あわ', 'ほう'},
 '為': {'い', 'ため', 'な', 'なり', 'なる'},
 '師': {'し', 'じ'},
 '博': {'はく', 'ばく', 'ぱく'},
 '灸': {'きゅう'},
 '意': {'い'},
 '階': {'かい', 'がい', 'きざはし'},
 '占': {'せん'},
 '詮': {'あき', 'せん'},
 '玄': {'くろ', 'げ', 'げん'},
 '岸': {'がん', 'きし', 'ぎし'},
 '橘': {'きつ', 'たちばな'},
 '研': {'け

In [38]:
#true_dict['言う']
#import csv

#my_dict = {"test": 1, "testing": 2}

#with open('falsedict.csv','w') as f:
#    w = csv.writer(f)
#    w.writerows(false_dict.items())

#with open('truedict.csv','w') as f:
#    w = csv.writer(f)
#    w.writerow(['Kanji','Pronunciation'])
#    w.writerows(true_dict.items())

import pickle

#true_dict.to_pickle()
pickle.dump( true_dict, open( "true.p", "wb" ) )
pickle.dump( kanji_pronunciations, open( "kanji.p", "wb" ) )

In [102]:
true_cnt = 0
false_cnt = 0
false_dict = {}
true_dict = {}

kanji_pronunciations = dict()
kanji_pron_counter = dict()

for _, line in temp.iterrows():
    kanji = line.vocab
    word = line.pronunciation
    prons, _ = recurs(kanji,word,[])
    if prons == None or len(prons) == 0:
        false_cnt += 1
        false_dict[kanji] = word
    else:
        for i in range(len(kanji)):
            kanji_pronunciations[kanji[i]] = kanji_pronunciations.get(kanji[i], set())
            kanji_pronunciations[kanji[i]].add(prons[i])
            kanji_pron_counter[kanji[i]] = kanji_pron_counter.get(kanji[i], Counter())
            kanji_pron_counter[kanji[i]][prons[i]] += 1
        true_cnt += 1
        true_dict[kanji] = prons

pickle.dump( true_dict, open( "vocab.p", "wb" ) )

In [92]:
true_cnt = 0
false_cnt = 0
false_dict = {}
true_dict = {}

kanji_pronunciations = dict()
kanji_pron_counter = dict()

for _, line in propernouns.iterrows():
    kanji = line.kanji
    word = line.pronunciation
    prons, _ = recurs(kanji,word,[])
    if prons == None or len(prons) == 0:
        false_cnt += 1
        false_dict[kanji] = word
    else:
        for i in range(len(kanji)):
            kanji_pronunciations[kanji[i]] = kanji_pronunciations.get(kanji[i], set())
            kanji_pronunciations[kanji[i]].add(prons[i])
            kanji_pron_counter[kanji[i]] = kanji_pron_counter.get(kanji[i], Counter())
            kanji_pron_counter[kanji[i]][prons[i]] += 1
        true_cnt += 1
        true_dict[kanji] = prons

pickle.dump( true_dict, open( "propernouns.p", "wb" ) )

In [15]:
true_dict.keys() & false_dict.keys()

{'公雄',
 '安紀',
 '岡豊',
 '環樹',
 '幾雄',
 '橘',
 '鍵山',
 '絢樹',
 '乙川八幡町',
 '恭香',
 '橘谷',
 '映見',
 '瑛志',
 '割田',
 '兼永',
 '干',
 '安立',
 '衣川',
 '開野',
 '恵麻',
 '晃聖',
 '金橋',
 '功生',
 '輝虎',
 '権現原',
 '館森',
 '暁青',
 '叶口',
 '光利',
 '結帆',
 '賀長',
 '愛瀬',
 '愛鈴',
 '英傑',
 '阿子嶋',
 '位子',
 '遠坂',
 '幹史',
 '華那子',
 '下河原',
 '玄珠',
 '光輝',
 '玄',
 '果菜',
 '下垣津',
 '愛梨',
 '丸岡町',
 '京都縦貫自動車道',
 '久利',
 '育代',
 '岩月',
 '金阪',
 '久芳',
 '下鷲谷',
 '原上',
 '覚',
 '更生',
 '遠入',
 '円里',
 '暁月',
 '岩谷川',
 '吾田',
 '孔人',
 '建雄',
 '安明',
 '康世',
 '江子',
 '鑑一',
 '橋詰',
 '慶雄',
 '幸聖',
 '橘尾',
 '央夏',
 '健富',
 '宇戸谷',
 '孝忠',
 '叶谷',
 '晃成',
 '鬼生田',
 '結太',
 '下埜',
 '具志頭',
 '久見',
 '晃世',
 '伊津子',
 '健将',
 '魚崎',
 '康甫',
 '幸丈',
 '陰山',
 '好雄',
 '叡樹',
 '吉本',
 '公英',
 '栄春',
 '華果',
 '寛秀',
 '炎',
 '英助',
 '佳之',
 '佳帆里',
 '謙',
 '空夜',
 '孝将',
 '嘉治',
 '賀',
 '間明田',
 '鞠郎',
 '鋭美',
 '吾記子',
 '惟二',
 '叶羽',
 '孝市',
 '伊香',
 '伊青',
 '佳津乃',
 '虎二',
 '絢三',
 '享子',
 '雅俊',
 '栄登',
 '一奈',
 '馨一',
 '栄崎',
 '匡一',
 '喜子',
 '貴子',
 '慧郎',
 '家井',
 '吟美',
 '光花',
 '顕徳',
 '威知',
 '啓行',
 '開花',
 '毅司',
 '我喜屋',
 '愛蓮',
 '岳世',
 '丸角',
 

In [16]:
false_dict.keys() | true_dict.keys()

{'綾依子',
 '穐津',
 'フロレス島',
 '館山下',
 '安積町南長久保',
 '永澤陽一',
 '甲盛',
 '駒隠沢',
 'スクイラチェ湾',
 '下津ふじ塚',
 '久世大築町',
 '駒形川',
 '垣入',
 '伽菜理',
 '郁李',
 '家次',
 '綾葵',
 '下薄葉',
 '旺文社',
 '希慧',
 '郁一郎',
 '玉木',
 '乙川八幡町',
 'ルアハ川',
 'ウムゲニ川',
 '奥ノ院',
 '稲吉靖司',
 '寛栄',
 '芽惟',
 '烏子沢',
 'ラカディブ諸島',
 '駒悠',
 '丸山朝日沢',
 '五月雄',
 '下品',
 '下鱗形町',
 '花登筺',
 '呉広',
 '伊賀野',
 '輝虎',
 'エイペックスゴルフ場',
 '岡井輝毅',
 '古烏帽子',
 '鼓動',
 '原崎政人',
 '弓削達',
 '一心子',
 '阿真子',
 '安加莉',
 'カウン沢',
 '河林',
 '金穂',
 '絵野',
 '加奈衣',
 '叶之助',
 '愛鈴',
 'ナリバ峰',
 '鶏峠山',
 '観勒',
 '角新田',
 '遠坂',
 'ヌプカクシュナイ川',
 '旭ヶ丘中町',
 '越後大島駅',
 '鮎川なおみ',
 '桐音子',
 '意智子',
 '金垣内',
 '晃信',
 '井久絵',
 '敬將',
 '花牟礼山',
 '桂令',
 '金刀比羅神社',
 '亜矢の',
 '孝二朗',
 '久世大築',
 '院下島',
 '丸駒温泉',
 '絵里南',
 '港屋',
 '訓菊',
 '安田与重郎',
 'たかの台',
 '江利田',
 '伊場',
 '五百旗',
 '一乗院',
 'ミスターアーサー２',
 '果涼',
 '阿多田島',
 '岳乗',
 '確允',
 '下鷲谷',
 '覚',
 'アンティトーラス山脈',
 '賀澤',
 '羽田埜',
 '閑馬川',
 '鴨島入江',
 'ボツワナ共和国',
 '卯之',
 '衣笠東御所ノ内町',
 '兼橋',
 '久実男',
 '貴音',
 '河北倫明',
 '岩谷川',
 '宜生',
 '暁月',
 '鴨居駅',
 '巌島神社',
 '吉田康夫',
 '賀蔵',
 '安ヶ川',
 '衣松',
 '安矢甫',
 '五郷町桃崎',
 '幸聖',

In [17]:
both_keys = true_dict.keys() & false_dict.keys()

In [18]:
len(both_keys)

10679

In [45]:
#jlpt[jlpt.Kanji.str.contains('亜')]
#jlpt[jlpt.Kanji.isnull()]
print(recurs('十一人','じゅういちにん',[]))
print(recurs('一人','ひとり',[]))
#print(jlpt)

(['じゅう', 'いち', 'にん'], True)
(['ひと', 'り'], True)
